# Solving Stokes equation with $(P_{1}^{CR}, P_{0})$

In [17]:
from skfem import *
import numpy as np
from skfem.visuals.matplotlib import draw, plot
from skfem.utils import solver_iter_krylov, solver_eigen_scipy, solver_iter_pcg
from skfem.helpers import dd, ddot, div, grad, d, prod, dot
from scipy.sparse.linalg import LinearOperator, minres
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import bmat
import dmsh
from skfem.assembly import BilinearForm, LinearForm
import scipy.sparse.linalg as spl

## Problem description

$$
\left\{\begin{array}{l}
- \Delta\boldsymbol{u} + \nabla p = \boldsymbol{f} \ in \  \Omega \\
\nabla\cdot\boldsymbol{u} = 0 \ in \  \Omega \\
u = 0 \ on \ \partial \Omega
\end{array}\right.
$$

where 

$$
\left\{\begin{array}{l}
\Delta \mathbf{u}=\sum_{i=1}^{N} \frac{\partial^{2} \mathbf{u}}{\partial x_{i} \partial x_{i}} \\
\nabla p=\left(\frac{\partial p}{\partial x_{1}}, \frac{\partial p}{\partial x_{2}}, \ldots, \frac{\partial p}{\partial x_{N}}\right) \\
\nabla \cdot \mathbf{u}=\sum_{1}^{N} \frac{\partial u_{i}}{\partial x_{i}}
\end{array}\right.
$$

`u` is the velocity vector and `p` is the pressure

Testing case in $\Omega=[0,1] \times [0,1]$:
$$
\left\{\begin{array}{l}
u1 = 10x^{2}(x - 1)^{2}y(y - 1)(2y - 1) \\
u2 = -10y^{2}(y - 1)^{2}x(x - 1)(2x - 1) \\
p = x^{2} - y^{2}
\end{array}\right.
$$

$$
\left(\begin{array}{cc}
A & -B \\
-B^{T} & 0
\end{array}\right)\left(\begin{array}{l}
U \\
P
\end{array}\right)=\left(\begin{array}{l}
F_{1} \\
0
\end{array}\right)
$$

## Adding $P_{1}^{CR}$ element

In [2]:
class ElementTriP1CR(ElementH1):
    
    facet_dofs = 1
    dim = 2
    maxdeg = 1
    dofnames = ['u']
    doflocs = np.array([[.5, 0.], [.5, .5], [0., .5]])
    mesh_type = MeshTri

    def lbasis(self, X, i):
        x, y = X

        if i == 0:
            phi = 1. - 2. * y
            dphi = np.array([0. * x, -2. + 0. * y])
            dphi = np.array([0. * x, -2. + 0. * y])
        elif i == 1:
            phi = 2. * x + 2. * y - 1.
            dphi = np.array([2. + 0. * x, 2. + 0. * y])
        elif i == 2:
            phi = 1. - 2. * x
            dphi = np.array([-2. + 0. * x, 0. * x])
        else:
            self._index_error()
        return phi, dphi

## Defining forms

In [3]:
@BilinearForm
def vector_laplace(u, v, w):
    '''
    a
    '''
    return ddot(grad(u), grad(v))


@BilinearForm
def divergence(u, v, w):
    '''
    b
    '''
    return div(u) * v


@LinearForm
def body_force(v, w):
    '''
    for f.*v
    '''
    x, y = w.x
    f1 = -10 * (12 * x**2 - 12 * x + 2) * y * (y - 1) * (2 * y - 1) - 10 * (
        x**2) * ((x - 1)**2) * (12 * y - 6) + 2 * x
    f2 = 10 * (12 * y**2 - 12 * y + 2) * x * (x - 1) * (2 * x - 1) + 10 * (y**2) * ((y - 1)**2) * (12 * x - 6) - 2 * y
    return f1 * v.value[0] + f2 * v.value[1]


@BilinearForm
def mass(u, v, w):
    '''
    c 
    '''
    return u * v * 0

## Defining exact value and $L_{2}$ error

In [4]:
def exactu(x, y):
    u1 = 10 * (x**2) * ((x - 1)**2) * y * (y - 1) * (2 * y - 1)
    u2 = -10 * (y**2) * ((y - 1)**2) * x * (x - 1) * (2 * x - 1)
    return u1, u2


@Functional
def L2Error_u(w):
    x, y = w.x
    u1, u2 = exactu(x, y)
    return (w.w[0] - u1)**2 + (w.w[1] - u2)**2


def exactp(x, y):
    return x**2 - y**2


@Functional
def L2Error_p(w):
    x, y = w.x
    return (w.w - exactp(x, y))**2

## Caculating error and convergence rate
> result: $h^{2}$ for `L2u` and $h^{1}$ for `L2p`

## MINRES

In [63]:
@BilinearForm
def penalty_1(u, v, w):
    global uu, vv, ww
    uu = u
    vv = v
    ww = w
    w_t = np.array([-w.n[1], w.n[0]])
    return -ddot(d(u), prod(w_t, w.n)) * dot(v, w_t)


@BilinearForm
def penalty_2(u, v, w):
    w_t = np.array([-w.n[1], w.n[0]])
    return -ddot(d(v), prod(w_t, w.n)) * dot(u, w_t)


@BilinearForm
def penalty_3(u, v, w):
    w_t = np.array([-w.n[1], w.n[0]])
    return (sigma / w.h) * dot(u, w_t) * dot(v, w_t)

In [58]:
w_t = np.array([-ww.n[1], ww.n[0]])
aaaa = -ddot(d(uu), prod(w_t, ww.n)) * dot(vv, w_t)
aaaa.shape

(64, 3)

In [56]:
du = d(uu)

In [57]:
du.shape

(2, 2, 64, 3)

In [46]:
vv.value.shape

(2, 64, 3)

In [48]:
dot(du[0], ww.n).shape

(64, 3)

In [51]:
(vv.value[0] * dot(du[0], ww.n)).shape

(64, 3)

In [60]:
(dot(du[0], ww.n) * vv[0] + dot(du[1], ww.n) * vv[1]).shape

(2, 2, 64, 3)

In [50]:
vv.value[0].shape

(64, 3)

In [53]:
dot(uu.value, vv.value).shape

(64, 3)

remove t

In [70]:
@BilinearForm
def penalty_1(u, v, w):
    du = d(u)
    return -(dot(du[0], w.n) * v.value[0] + dot(du[1], w.n) * v.value[1])


@BilinearForm
def penalty_2(u, v, w):
    dv = d(v)
    return -(dot(dv[0], w.n) * u.value[0] + dot(dv[1], w.n) * u.value[1])


@BilinearForm
def penalty_3(u, v, w):
    return (sigma / w.h) * dot(u, v)

prod(wt, wn)

In [68]:
@BilinearForm
def penalty_1(u, v, w):
    w_t = np.array([-w.n[1], w.n[0]])
    # return -dot(u, v)
    return -ddot(d(u), prod(w_t, w.n)) * dot(v, w_t) - ddot(d(u), prod(w.n, w.n)) * dot(v, w.n)


@BilinearForm
def penalty_2(u, v, w):
    w_t = np.array([-w.n[1], w.n[0]])
    return -ddot(d(v), prod(w_t, w.n)) * dot(u, w_t) - ddot(d(v), prod(w.n, w.n)) * dot(u, w.n)


@BilinearForm
def penalty_3(u, v, w):
    w_t = np.array([-w.n[1], w.n[0]])
    return (sigma / w.h) * (dot(u, w_t) * dot(v, w_t) + dot(u, w.n) * dot(v, w.n))

In [30]:
def normal_boundary(basis):
    '''
    Input basis
    ----------------
    Return D for boundary conditions for u^n
    ----------------
    Note: u^1 here stands for the first component of u and u^2 for the second
    '''

    dofs = basis.find_dofs({
        'left': m.facets_satisfying(lambda x: x[0] == 0),
        'right': m.facets_satisfying(lambda x: x[0] == 1),
        'top': m.facets_satisfying(lambda x: x[1] == 1),
        'buttom': m.facets_satisfying(lambda x: x[1] == 0)
    })

    D = np.concatenate((dofs['left'].facet['u^1'], dofs['right'].facet['u^1'],
                        dofs['top'].facet['u^2'], dofs['buttom'].facet['u^2']))
    return D

In [69]:
formerL2p = 1
currentL2p = 1
formerL2u = 1
currentL2u = 1
sigma = 5
for i in range(5):
    # mesh = MeshTri()
    m = MeshTri.init_symmetric()
    m.refine(i)
    element = {'u': ElementVectorH1(ElementTriP1CR()), 'p': ElementTriP0()}
    basis = {
        variable: InteriorBasis(m, e, intorder=4)
        for variable, e in element.items()
    }  # intorder: integration order for quadrature

    
    fbasis = FacetBasis(m, element['u'], intorder=4)
    
    p1 = asm(penalty_1, fbasis) 
    p2 = asm(penalty_2, fbasis) 
    p3 = asm(penalty_3, fbasis) 
    P = p1 + p2 + p3
    
    A = asm(vector_laplace, basis['u']) + P
    B = asm(divergence, basis['u'], basis['p'])
    C = asm(mass, basis['p'])

    K = bmat([[A, -B.T], [-B, 1e-6 * C]],
             'csr')  # get the sparse format of the result by 'csr'
    f = np.concatenate([asm(body_force, basis['u']), np.zeros(B.shape[0])])
    
    up, _ = spl.minres(K, f)
    
    # up = solve(*condense(K, f, D=normal_boundary(basis['u'])), solver=solver_iter_krylov(spl.minres, tol=1e-11, show=False))
    # up = solve(*condense(K, f, D=basis['u'].find_dofs()), solver=solver_iter_krylov(spl.minres, tol=1e-11, show=False))

    uh, ph = np.split(up, [A.shape[0]])
    # p = exactp(basis['p'].doflocs[0], basis['p'].doflocs[1])
    # print((np.sqrt(np.sum((p-ph)**2)))/len(ph))
    P = basis['p'].interpolate(ph).value
    L2p = np.sqrt(L2Error_p.assemble(basis['p'], w=P))
    U = basis['u'].interpolate(uh).value
    L2u = np.sqrt(L2Error_u.assemble(basis['u'], w=U))
    currentL2p = L2p
    currentL2u = L2u
    if i != 0:
        print('2^-' + str(i + 1) + ' case')
        print('L2u Error:', L2u)
        print('L2p Error', L2p)
        print('L2u rate', -np.log2(currentL2u / formerL2u))
        print('L2p rate', -np.log2(currentL2p / formerL2p))
    formerL2p = L2p
    formerL2u = L2u

2^-2 case
L2u Error: 0.05382465790162129
L2p Error 0.3781500933831148
L2u rate 1.3198847171986847
L2p rate 0.28726162640826064
2^-3 case
L2u Error: 0.033629465851170964
L2p Error 0.28690193222504773
L2u rate 0.6785413807453843
L2p rate 0.39840129079581366
2^-4 case
L2u Error: 0.024693556375375965
L2p Error 0.23725374324423118
L2u rate 0.4455912352729321
L2p rate 0.27412683500678947
2^-5 case
L2u Error: 0.03583605437290778
L2p Error 0.2909170439813212
L2u rate -0.5372771733627266
L2p rate -0.2941769728254092


In [71]:
formerL2p = 1
currentL2p = 1
formerL2u = 1
currentL2u = 1
sigma = 5
for i in range(5):
    # mesh = MeshTri()
    m = MeshTri.init_symmetric()
    m.refine(i)
    element = {'u': ElementVectorH1(ElementTriP1CR()), 'p': ElementTriP0()}
    basis = {
        variable: InteriorBasis(m, e, intorder=4)
        for variable, e in element.items()
    }  # intorder: integration order for quadrature

    
    fbasis = FacetBasis(m, element['u'], intorder=4)
    
    p1 = asm(penalty_1, fbasis) 
    p2 = asm(penalty_2, fbasis) 
    p3 = asm(penalty_3, fbasis) 
    P = p1 + p2 + p3
    
    A = asm(vector_laplace, basis['u']) + P
    B = asm(divergence, basis['u'], basis['p'])
    C = asm(mass, basis['p'])

    K = bmat([[A, -B.T], [-B, 1e-6 * C]],
             'csr')  # get the sparse format of the result by 'csr'
    f = np.concatenate([asm(body_force, basis['u']), np.zeros(B.shape[0])])
    
    up, _ = spl.minres(K, f)
    
    # up = solve(*condense(K, f, D=normal_boundary(basis['u'])), solver=solver_iter_krylov(spl.minres, tol=1e-11, show=False))
    # up = solve(*condense(K, f, D=basis['u'].find_dofs()), solver=solver_iter_krylov(spl.minres, tol=1e-11, show=False))

    uh, ph = np.split(up, [A.shape[0]])
    # p = exactp(basis['p'].doflocs[0], basis['p'].doflocs[1])
    # print((np.sqrt(np.sum((p-ph)**2)))/len(ph))
    P = basis['p'].interpolate(ph).value
    L2p = np.sqrt(L2Error_p.assemble(basis['p'], w=P))
    U = basis['u'].interpolate(uh).value
    L2u = np.sqrt(L2Error_u.assemble(basis['u'], w=U))
    currentL2p = L2p
    currentL2u = L2u
    if i != 0:
        print('2^-' + str(i + 1) + ' case')
        print('L2u Error:', L2u)
        print('L2p Error', L2p)
        print('L2u rate', -np.log2(currentL2u / formerL2u))
        print('L2p rate', -np.log2(currentL2p / formerL2p))
    formerL2p = L2p
    formerL2u = L2u

2^-2 case
L2u Error: 0.05382465790162129
L2p Error 0.3781500933831148
L2u rate 1.3198847171986847
L2p rate 0.28726162640826064
2^-3 case
L2u Error: 0.033629465851170964
L2p Error 0.28690193222504773
L2u rate 0.6785413807453843
L2p rate 0.39840129079581366
2^-4 case
L2u Error: 0.024693556375375965
L2p Error 0.23725374324423118
L2u rate 0.4455912352729321
L2p rate 0.27412683500678947
2^-5 case
L2u Error: 0.03583605437290778
L2p Error 0.2909170439813212
L2u rate -0.5372771733627266
L2p rate -0.2941769728254092


# Original

In [10]:
formerL2p = 1
currentL2p = 1
formerL2u = 1
currentL2u = 1
for i in range(5):
    # mesh = MeshTri()
    mesh = MeshTri.init_symmetric()
    mesh.refine(i)
    element = {'u': ElementVectorH1(ElementTriP1CR()), 'p': ElementTriP0()}
    basis = {
        variable: InteriorBasis(mesh, e, intorder=4)
        for variable, e in element.items()
    }  # intorder: integration order for quadrature


    
    A = asm(vector_laplace, basis['u'])
    B = asm(divergence, basis['u'], basis['p'])
    C = asm(mass, basis['p'])

    K = bmat([[A, -B.T], [-B, 1e-6 * C]],
             'csr')  # get the sparse format of the result by 'csr'
    f = np.concatenate([asm(body_force, basis['u']), np.zeros(B.shape[0])])

    up = solve(*condense(K, f, D=basis['u'].find_dofs()), solver=solver_iter_krylov(spl.minres, tol=1e-13, show=False))
    # up = solve(*condense(K, f, D=basis['u'].find_dofs()), solver=solver_iter_krylov(spl.minres, tol=1e-11, show=False))

    uh, ph = np.split(up, [A.shape[0]])
    # p = exactp(basis['p'].doflocs[0], basis['p'].doflocs[1])
    # print((np.sqrt(np.sum((p-ph)**2)))/len(ph))
    P = basis['p'].interpolate(ph).value
    L2p = np.sqrt(L2Error_p.assemble(basis['p'], w=P))
    U = basis['u'].interpolate(uh).value
    L2u = np.sqrt(L2Error_u.assemble(basis['u'], w=U))
    currentL2p = L2p
    currentL2u = L2u
    if i != 0:
        print('2^-' + str(i + 1) + ' case')
        print('L2u Error:', L2u)
        print('L2p Error', L2p)
        print('L2u rate', -np.log2(currentL2u / formerL2u))
        print('L2p rate', -np.log2(currentL2p / formerL2p))
    formerL2p = L2p
    formerL2u = L2u

2^-2 case
L2u Error: 0.022877238694641165
L2p Error 0.191058437310352
L2u rate 0.9693896038730285
L2p rate 0.6306275386753827
2^-3 case
L2u Error: 0.00684063432302261
L2p Error 0.08433594314357606
L2u rate 1.7417109122646732
L2p rate 1.1797944401199743
2^-4 case
L2u Error: 0.0019121039085281534
L2p Error 0.039873993093293404
L2u rate 1.838969185407123
L2p rate 1.0806995364745338
2^-5 case
L2u Error: 0.0004969343447816387
L2p Error 0.019221536012063157
L2u rate 1.944033765232154
L2p rate 1.0527244601758676
